In [1]:
import syft as sy
import torch as th
from syft.core.plan.plan_builder import make_plan
from syft.lib.python.collections import OrderedDict
import collections

In [2]:
alice = sy.VirtualMachine()
alice_client = alice.get_root_client()

[2021-03-24T10:19:34.145871+0800][CRITICAL][logger]][872] Skipping torchvision.torchvision.transforms.functional.adjust_sharpness not supported in 0.8.1
[2021-03-24T10:19:34.146279+0800][CRITICAL][logger]][872] Skipping torchvision.torchvision.transforms.functional.autocontrast not supported in 0.8.1
[2021-03-24T10:19:34.146533+0800][CRITICAL][logger]][872] Skipping torchvision.torchvision.transforms.functional.equalize not supported in 0.8.1
[2021-03-24T10:19:34.146826+0800][CRITICAL][logger]][872] Skipping torchvision.torchvision.transforms.functional.invert not supported in 0.8.1
[2021-03-24T10:19:34.147087+0800][CRITICAL][logger]][872] Skipping torchvision.torchvision.transforms.functional.posterize not supported in 0.8.1
[2021-03-24T10:19:34.147354+0800][CRITICAL][logger]][872] Skipping torchvision.torchvision.transforms.functional.solarize not supported in 0.8.1


In [3]:
# Linear
fc = th.nn.Linear(4,2)

# send
fc_ptr = fc.send(alice_client)
print(f"----fc_ptr----\n{fc_ptr}\n")

# remote call
res_ptr = fc_ptr(th.rand([1,4]))
print(f"----res_ptr----\n{res_ptr}\n")
print(f"----res_ptr.get()----\n{res_ptr.get()}\n")

# remote update state dict
sd2 = OrderedDict(th.nn.Linear(4,2).state_dict())
sd2_ptr = sd2.send(alice_client)
fc_ptr.load_state_dict(sd2_ptr)

# get
print(f"----fc_ptr.get().state_dict()----\n{fc_ptr.get().state_dict()}\n")
print(f"----sd2----\n{sd2}\n")

----fc_ptr----

----res_ptr----

----res_ptr.get()----
tensor([[0.8382, 0.8850]], requires_grad=True)

----fc_ptr.get().state_dict()----
OrderedDict([('weight', tensor([[-0.2090, -0.0208,  0.4798, -0.2642],
        [ 0.2569, -0.0775,  0.0796, -0.4728]])), ('bias', tensor([-0.2466,  0.0769]))])

----sd2----
OrderedDict([('weight', tensor([[-0.2090, -0.0208,  0.4798, -0.2642],
        [ 0.2569, -0.0775,  0.0796, -0.4728]])), ('bias', tensor([-0.2466,  0.0769]))])



/mnt/d/github/PySyft/src/syft/lib/torch/uppercase_tensor.py:30: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  grad = getattr(obj, "grad", None)


In [4]:
# ReLU
relu = th.nn.ReLU(inplace=True)

# send
relu_ptr = relu.send(alice_client)
print(f"----relu_ptr----\n{relu_ptr}\n")

# remote call
res_ptr = relu_ptr(th.rand([1,4]))
print(f"----res_ptr----\n{res_ptr}\n")
print(f"----res_ptr.get()----\n{res_ptr.get()}\n")

# get
print(f"----relu_ptr.get()----\n{relu_ptr.get()}\n")

[2021-03-24T10:19:34.277944+0800][CRITICAL][logger]][872] <class 'syft.core.store.store_memory.MemoryStore'> __delitem__ error <UID: 348a2be75cda41058bf18e4fcbdce253>.


----relu_ptr----

----res_ptr----

----res_ptr.get()----
tensor([[0.8725, 0.4835, 0.6143, 0.2234]])

----relu_ptr.get()----
ReLU(inplace=True)



In [5]:
# Sequential
seq = th.nn.Sequential()
seq.add_module("fc1", th.nn.Linear(4,2))
seq.add_module("fc2", th.nn.Linear(2,1))

# send
seq_ptr = seq.send(alice_client)
print(f"----seq_ptr----\n{seq_ptr}\n")

# remote call
res_ptr = seq_ptr(th.rand([1,4]))
print(f"----res_ptr----\n{res_ptr}\n")
print(f"----res_ptr.get()----\n{res_ptr.get()}\n")

# remote update state dict
sd2 = OrderedDict(
    th.nn.Sequential(
        collections.OrderedDict([
            ("fc1", th.nn.Linear(4,2)),
            ("fc2", th.nn.Linear(2,1))
        ])
    ).state_dict()
)
sd2_ptr = sd2.send(alice_client)
seq_ptr.load_state_dict(sd2_ptr)

# get
print(f"----seq_ptr.get().state_dict()----\n{seq_ptr.get().state_dict()}\n")
print(f"----sd2----\n{sd2}\n")

[2021-03-24T10:19:34.304693+0800][CRITICAL][logger]][872] <class 'syft.core.store.store_memory.MemoryStore'> __delitem__ error <UID: a782634d5cdb4f149dc15f385a041753>.


----seq_ptr----

----res_ptr----

----res_ptr.get()----
tensor([[0.1151]], requires_grad=True)

----seq_ptr.get().state_dict()----
OrderedDict([('fc1.weight', tensor([[ 0.1484, -0.2039, -0.4905,  0.2857],
        [-0.2914, -0.0684,  0.1452,  0.0238]])), ('fc1.bias', tensor([-0.2603, -0.0075])), ('fc2.weight', tensor([[-0.3948,  0.3055]])), ('fc2.bias', tensor([-0.5001]))])

----sd2----
OrderedDict([('fc1.weight', tensor([[ 0.1484, -0.2039, -0.4905,  0.2857],
        [-0.2914, -0.0684,  0.1452,  0.0238]])), ('fc1.bias', tensor([-0.2603, -0.0075])), ('fc2.weight', tensor([[-0.3948,  0.3055]])), ('fc2.bias', tensor([-0.5001]))])



In [6]:
# user defined model
class M(th.nn.Module):
    def __init__(self):
        super(M, self).__init__()
        self.fc1 = th.nn.Linear(4,2)
        self.fc2 = th.nn.Linear(2,1)
        
    def forward(self,x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x
    
    @property
    def forward_Plan(self):
        @make_plan
        def remote_forward(
            x=th.rand([4,4]), 
            fc1 = self.fc1,
            fc2 = self.fc2
        ):
            x = fc1(x)
            x = fc2(x)
            return x
        return remote_forward
        
m = M()

# send
m_ptr = m.send(alice_client)
print(f"----m_ptr----\n{m_ptr}\n")

# TODO: remote call
# res_ptr = m_ptr(x=th.rand([1,25]))
# print(f"----res_ptr----\n{res_ptr}\n")
# print(f"----res_ptr.get()----\n{res_ptr.get()}\n")

# get
print(f"----m_ptr.get()----\n{m_ptr.get()}\n")

----m_ptr----

----m_ptr.get()----
Module(
  (fc1): Linear(in_features=4, out_features=2, bias=True)
  (fc2): Linear(in_features=2, out_features=1, bias=True)
)

